# Extracting catalogue content from MOLES

In [1]:
import requests
url = "https://api.catalogue.ceda.ac.uk/api/v2/observations/?format=json&?publicationStatus__in=published,citable"

In [41]:
resp = requests.get(url).json()

In [42]:
next_doc = resp["next"]

In [43]:
results = resp["results"]

In [44]:
len(results)

100

In [45]:
res = results[0]

In [46]:
fields = ['ob_id', 'uuid', 'title', 'abstract', 'keywords', 'publicationState', 'dataPublishedTime', 'doiPublishedTime', 
 'updateFrequency', 'status', 'result_field', 'timePeriod', 'geographicExtent', 'nonGeographicFlag', 'phenomena', 
 'dataLineage', 'removedDataTime', 'removedDataReason', 'language', 'identifier_set', 'projects', 
 'observationcollection_set', 'responsiblepartyinfo_set', 'procedureAcquisition', 'procedureCompositeProcess', 
 'procedureComputation']

to_keep = ["uuid", "title", "abstract", "keywords", "timePeriod", "geographicExtent",
           "phenomena", "projects"]

In [47]:
for key in to_keep:
    print(key, "-->", res.get(key, ""))

uuid --> 3ca88ef5aca50e29b257ffaff3a047ab
title --> Met Office: 64 MHz ST vertical wind profiler measurements at South Uist, UK (2004 - 2012)
abstract --> The Met Office's 64 MHz ST (stratosphere-troposphere) wind profiler was installed at the Met Office South Uist range meteorological station, Hebrides, in May 2003. The radar complements the MST radar located near Abersywyth, providing greater vertical coverage than the other wind profiling radars in the Met Office's UK "Operational Upper Air Network." Operating at 64 Mhz, the South Uist 64 Mhz radar runs typically sounds from 1 to 13 km with a heigh resolution of either 150 or 400m, depending on the mode of operation. The site is located at 57.353 N, 7.375 W and at a height of 4 m above mean sea-level and has a WMO id of 03019. It had a beam angle of 15.0 degrees and is operated with an averaging period of 30 minutes.

This dataset contains vertical wind profiles from the 64MHz wind profiler located at Camborne.
keywords --> Met Offi

In [48]:
def decode_time(url):
    resp = requests.get(url).json()
    return resp.get("startTime", ""), resp.get("endTime", "")

decode_time(res["timePeriod"])

('2004-05-30T23:00:00', '2012-05-10T23:59:59')

In [49]:
def decode_space(url):
    resp = requests.get(url).json()
    smap = {"east": "eastBoundLongitude", "north": "northBoundLatitude",
            "west": "westBoundLongitude", "south": "southBoundLatitude"}
    return {key: resp.get(exkey, "") for key, exkey in smap.items()}

decode_space(res["geographicExtent"])

{'east': -7.375, 'north': 57.353, 'west': -7.375, 'south': 57.353}

In [50]:
def decode_projects(urls):
    projects = [requests.get(url).json().get("title", "") for url in urls]
    return [proj for proj in projects if proj]

decode_projects(res["projects"])

['Met Office Wind Profiler data (1998-onwards)']

In [51]:
def decode_phenomena(urls):
    phenomena = [requests.get(url).json().get("names", []) for url in urls]
    return sorted([name.get("name") for phen in phenomena for name in phen if not name.get("name", "").startswith("http")])

decode_phenomena(res["phenomena"])

['Upper Level Winds', 'Wind Direction', 'Windspeed']